In [1]:
import pybaseball
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
batter_info = pybaseball.statcast('2024-04-03', '2024-04-20')
batter_info.head()

This is a large query, it may take a moment to complete


  0%|          | 0/18 [00:00<?, ?it/s]c:\Users\yjain\Desktop\Personal Work\moonshot\.venv\Lib\site-packages\pybaseball\datahelpers\postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
  6%|▌         | 1/18 [00:02<00:40,  2.36s/it]c:\Users\yjain\Desktop\Personal Work\moonshot\.venv\Lib\site-packages\pybaseball\datahelpers\postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
 11%|█         | 2/18 [00:02<00:19,  1.19s/it]c:\Users\yjain\Desktop\Personal Work\moonshot\.venv\Lib\site-packages\pybaseball\datahelpers\postprocessing.py:59: Fut

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle,attack_angle,attack_direction,swing_path_tilt,intercept_ball_minus_batter_pos_x_inches,intercept_ball_minus_batter_pos_y_inches
3203,CH,2024-04-20,90.1,-1.49,5.58,"Iglesias, Raisel",694671,628452,strikeout,swinging_strike,...,1,2.54,1.53,1.53,37.2,8.811087,-6.044923,39.22471,26.790648,31.434949
3222,FF,2024-04-20,96.7,-1.14,5.78,"Iglesias, Raisel",694671,628452,NaN,ball,...,1,1.17,0.35,0.35,39.0,<NA>,<NA>,<NA>,<NA>,<NA>
3340,SL,2024-04-20,84.3,-1.51,5.52,"Iglesias, Raisel",694671,628452,NaN,ball,...,1,3.18,-1.13,-1.13,35.2,<NA>,<NA>,<NA>,<NA>,<NA>
3463,CH,2024-04-20,89.5,-1.69,5.51,"Iglesias, Raisel",694671,628452,NaN,foul,...,1,2.43,1.31,1.31,32.7,20.160104,-24.888211,29.967692,20.944166,41.321476
3592,SL,2024-04-20,84.5,-1.78,5.53,"Iglesias, Raisel",694671,628452,NaN,called_strike,...,1,3.0,-0.37,-0.37,32.7,<NA>,<NA>,<NA>,<NA>,<NA>


In [3]:
shortened_data = batter_info[['batter', 'pitch_type', 'plate_x', 'plate_z', 'events', 'release_speed', 'release_pos_x', 'launch_speed', 'launch_angle', 'effective_speed', 'release_spin_rate', 'release_extension']]
pruned_data = shortened_data.dropna()
print(pruned_data.shape)
pruned_data.head()




(11714, 12)


,batter,pitch_type,plate_x,plate_z,events,release_speed,release_pos_x,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension
4539,663993,CH,-0.56,1.88,field_out,89.0,-1.77,97.4,17,88.6,1864,6.2
2439,663586,FF,0.01,1.79,field_out,93.2,-0.24,107.4,22,92.4,2410,5.8
2870,671739,CH,-0.72,1.51,single,88.2,-0.51,86.3,13,87.5,1745,5.8
3794,641645,CH,-0.76,1.89,field_out,87.5,-0.39,93.8,-17,86.9,1712,5.7
4057,672284,SL,1.0,2.78,field_out,88.4,-0.46,63.6,25,87.9,2604,5.7


In [14]:
batter_enc = LabelEncoder()
pruned_data['batter_id'] = batter_enc.fit_transform(pruned_data['batter']) # We may not need this as 'batter' is already a label
pitch_enc = LabelEncoder()
pruned_data['pitch_type_id'] = pitch_enc.fit_transform(pruned_data['pitch_type'])
outcome_enc = LabelEncoder()
pruned_data['outcome_id'] = outcome_enc.fit_transform(pruned_data['events'])

NUM_BATTERS = pruned_data['batter_id'].nunique()
NUM_PITCHES = pruned_data['pitch_type_id'].nunique()
NUM_OUTCOMES = pruned_data['outcome_id'].nunique()

scaler = StandardScaler()
pruned_data[['release_speed', 'release_pos_x', 'plate_x', 'plate_z', 'launch_speed', 'launch_angle', 'effective_speed', 'release_spin_rate', 'release_extension']] = scaler.fit_transform(pruned_data[['release_speed', 'release_pos_x', 'plate_x', 'plate_z', 'launch_speed', 'launch_angle', 'effective_speed', 'release_spin_rate', 'release_extension']].astype(float))
pruned_data.head()

pruned_data = pruned_data[pruned_data['events'] != 'catcher_interf']

print(pruned_data['outcome_id'].value_counts())
print(pruned_data['events'].value_counts())

print(NUM_OUTCOMES)


outcome_id
3     6951
11    2479
0      720
8      451
6      347
7      317
10     121
2      115
12      62
9       41
4       40
1       35
5       34
Name: count, dtype: int64
events
field_out                    6951
single                       2479
double                        720
home_run                      451
force_out                     347
grounded_into_double_play     317
sac_fly                       121
field_error                   115
triple                         62
sac_bunt                       41
fielders_choice                40
double_play                    35
fielders_choice_out            34
Name: count, dtype: int64
13


In [15]:
class SuperDataSet(Dataset):
    def __init__(self, df):
        self.df = df.reset_index(drop=True)
    def __getitem__(self, index):
        row = self.df.loc[index]
        batter_id = int(row['batter_id'])
        pitch_type_id = int(row['pitch_type_id'])
        features = torch.tensor(row[['release_speed', 'release_pos_x', 'plate_x', 'plate_z', 'launch_speed', 'launch_angle', 'effective_speed', 'release_spin_rate', 'release_extension']].values.astype(np.float32), dtype=torch.float32)
        y = int(row['outcome_id'])
        return batter_id, pitch_type_id, features, y
    def __len__(self):
        return len(self.df)




train_df, test_df = train_test_split(pruned_data, test_size=0.2, random_state=42, stratify=pruned_data['outcome_id'])

train_dataset = SuperDataSet(train_df)
test_dataset = SuperDataSet(test_df)

print(f"Train dataset size: {len(train_dataset)}")
print(train_dataset[1])
print(f"Test dataset size: {len(test_dataset)}")
print(test_dataset[1])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=24, shuffle=False)



Train dataset size: 9370
(404, 0, tensor([ 0.6163, -0.7102,  0.2110, -2.0018, -0.9332, -0.6822,  0.6348, -1.4158,
         0.2321]), 3)
Test dataset size: 2343
(47, 9, tensor([-2.0611,  1.3769,  1.0002, -1.5801, -0.0183, -0.1236, -1.7993,  0.9627,
         1.2920]), 3)


In [16]:
class SuperModel(nn.Module):
    def __init__(self, num_batters, num_pitch_types, num_input_data_types, output_dim, batter_embedding=10, pitch_embedding=5, hidden_dim=64): #Mess around with batter_embedding, pitch_embedding, hidden_dim
        super().__init__()
        self.batter_embedding = nn.Embedding(num_batters, batter_embedding)
        self.pitch_embedding = nn.Embedding(num_pitch_types, pitch_embedding)
        self.linear1 = nn.Linear(num_input_data_types, 32)
        self.linear2 = nn.Linear(32, 16)

        self.fc1 = nn.Linear(batter_embedding + pitch_embedding + 16, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
    def forward(self, batter_id, pitch_id, input_data):
        b = self.batter_embedding(batter_id)
        p = self.pitch_embedding(pitch_id) 
        i = self.linear1(input_data)
        i = self.relu(i)
        i = self.linear2(i)
        i = self.relu(i)
        x = torch.cat([b, p, i], dim=1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x # {'foul': 0.1, 'single': 0.3, ...}

In [17]:

model_t = SuperModel(num_batters=NUM_BATTERS, num_pitch_types=NUM_PITCHES, num_input_data_types=9, output_dim=NUM_OUTCOMES).to('cuda')
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_t.parameters(), lr=0.001)
print(model_t)


SuperModel(
  (batter_embedding): Embedding(426, 10)
  (pitch_embedding): Embedding(15, 5)
  (linear1): Linear(in_features=9, out_features=32, bias=True)
  (linear2): Linear(in_features=32, out_features=16, bias=True)
  (fc1): Linear(in_features=31, out_features=64, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=64, out_features=13, bias=True)
)


In [18]:
for epoch in range(10):
    model_t.train()
    for i, (batter_id, pitch_type_id, features, y) in enumerate(train_loader):
        batter_id = batter_id.to('cuda').long()
        pitch_type_id = pitch_type_id.to('cuda').long()
        features = features.to('cuda')
        y = y.to('cuda')
        
        print(type(batter_id), type(pitch_type_id), type(features), type(y))
        if hasattr(model_t, 'batter_embedding'):
            max_bat = batter_id.max().item()
            if max_bat >= model_t.batter_embedding.num_embeddings or batter_id.min().item() < 0:
                print(f"bad batch{i}")
                raise ValueError(f"batter_id contains values out of range: {batter_id.min().item()} to {max_bat}, but embedding size is {model_t.batter_embedding.num_embeddings}")
        
        if hasattr(model_t, "pitch_embedding"):
            max_idx = int(pitch_type_id.max().item())
            if max_idx >= model_t.pitch_embedding.num_embeddings or pitch_type_id.min().item() < 0:
                print("Bad pitch_type index in batch", i, "min/max:", pitch_type_id.min().item(), pitch_type_id.max().item())
                raise RuntimeError("pitch_type_id out of range for embedding")

        # If using CrossEntropyLoss:
        if y.min().item() < 0 or y.max().item() > 13:
            print("Invalid target values in batch", i, "min/max:", y.min().item(), y.max().item())
            raise RuntimeError("target class index out of range for CrossEntropyLoss")

        # basic shape/dtype print (once)
        if i == 0 and epoch == 0:
            print("Shapes:", batter_id.shape, pitch_type_id.shape, features.shape, y.shape)
            print("Dtypes:", batter_id.dtype, pitch_type_id.dtype, features.dtype, y.dtype)

        logits = model_t(batter_id, pitch_type_id, features)
        loss_value = loss(logits, y)
        optimizer.zero_grad()
        loss_value.backward()
        optimizer.step()
        print(f"Epoch {epoch+1}, Loss: {loss_value.item()}")



<class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'>
Shapes: torch.Size([16]) torch.Size([16]) torch.Size([16, 9]) torch.Size([16])
Dtypes: torch.int64 torch.int64 torch.float32 torch.int64
Epoch 1, Loss: 2.5141758918762207
<class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'>
Epoch 1, Loss: 2.4920096397399902
<class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'>
Epoch 1, Loss: 2.49658465385437
<class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'>
Epoch 1, Loss: 2.472123622894287
<class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'>
Epoch 1, Loss: 2.4559803009033203
<class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'>
Epoch 1, Loss: 2.4063916206359863
<class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'>
Epoch 1, Loss: 2.3